<a href="https://colab.research.google.com/github/askmuhsin/colab_selenium/blob/main/awb_tracker.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# ! rm -rf ./colab_selenium
! git clone https://github.com/askmuhsin/colab_selenium

! chmod +x ./colab_selenium/install_selenium.sh
! ./colab_selenium/install_selenium.sh

Cloning into 'colab_selenium'...
remote: Enumerating objects: 11, done.
remote: Counting objects: 100% (11/11), done.
remote: Compressing objects: 100% (7/7), done.
remote: Total 11 (delta 2), reused 8 (delta 2), pack-reused 0
Unpacking objects: 100% (11/11), 3.16 KiB | 810.00 KiB/s, done.
Executing: /tmp/apt-key-gpghome.QymtSM8WFj/gpg.1.sh --keyserver keyserver.ubuntu.com --recv-keys DCC9EFBF77E11517
gpg: key DCC9EFBF77E11517: "Debian Stable Release Key (10/buster) <debian-release@lists.debian.org>" not changed
gpg: Total number processed: 1
gpg:              unchanged: 1
Executing: /tmp/apt-key-gpghome.Xu27RLjM6x/gpg.1.sh --keyserver keyserver.ubuntu.com --recv-keys 648ACFD622F3D138
gpg: key DC30D7C23CBBABEE: "Debian Archive Automatic Signing Key (10/buster) <ftpmaster@debian.org>" not changed
gpg: Total number processed: 1
gpg:              unchanged: 1
Executing: /tmp/apt-key-gpghome.jIsqc9f8BR/gpg.1.sh --keyserver keyserver.ubuntu.com --recv-keys 112695A0E562B32A
gpg: key 4DFAB270

In [ ]:
from colab_selenium.utils import setup_driver
from bs4 import BeautifulSoup
import json
import time

In [ ]:
def scrape_poslaju(awb, driver):
    url = f"https://app.easyparcel.com/my/en/track/details/?awb={awb}&courier=poslaju"
    print('Scrapping -- ', url)

    driver.get(url)
    time.sleep(3)
    html = driver.page_source
    soup = BeautifulSoup(html, "html.parser")

    statuspoints = soup.select('.statuspoint__content')
    status_list = []
    for sp in statuspoints:
        status = sp.select_one('strong').text
        location = sp.select_one('.hint').text

        time_elem = sp.parent.select_one('.statuspoint__time')
        status_date = time_elem.select_one('strong').text
        status_time = time_elem.select_one('.hint').text
        
        status_dict = {
            'status': status,
            'location': location,
            'date': status_date,
            'time': status_time,
        }
        
        status_list.append(status_dict)

    return status_list


In [ ]:
driver = setup_driver()

/usr/local/lib/python3.9/dist-packages/selenium_profiles/scripts/profiles.py:74: UserWarning: Might be more likely to get detected with sandbox set to False!
  warnings.warn('Might be more likely to get detected with sandbox set to False!')


In [ ]:
awbs = ["EN706068813MY"]
for awb in awbs:
    status_list = scrape_poslaju(awb, driver)
    print(json.dumps(status_list, indent=1))

Scrapping --  https://app.easyparcel.com/my/en/track/details/?awb=EN706068813MY&courier=poslaju
[
 {
  "status": "Item processed",
  "location": "HUB Parcel Domestik Shah Alam",
  "date": "2023-03-04",
  "time": "03:05 AM"
 },
 {
  "status": "Item dispatched out",
  "location": "Pusat POS Laju Sungai Petani",
  "date": "2023-03-03",
  "time": "18:05 PM"
 },
 {
  "status": "Item picked up",
  "location": "Pusat POS Laju Sungai Petani",
  "date": "2023-03-03",
  "time": "12:29 PM"
 }
]
